<a href="https://colab.research.google.com/github/TimHBSWFL/UCSD-ML-Capstone/blob/main/baseline_business_attributes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, classification_report, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
filename = "FL_Restaurants_Business Attributes" + ".csv"
directory = '/content/drive/My Drive/Capstone Data Collection/'

path = directory + filename

chunk_iterator = pd.read_csv(path, chunksize=10000)

chunks = []

for chunk in chunk_iterator:
  chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)
df.shape

(8721, 16)

In [41]:
df = df.drop(['business_id', 'name', 'address', 'city_original', 'latitude', 'longitude', 'zip_code', 'hours', 'state'], axis=1)
df.head()

,postal_code,stars,review_count,is_open,attributes,categories,city_updated
0,33602,4.0,10,1,"{'Alcohol': ""'none'"", 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks",Tampa
1,33771,4.5,100,0,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","Food, Delis, Italian, Bakeries, Restaurants",Largo
2,33618,4.0,23,0,"{'BusinessParking': ""{'garage': False, 'street...","Restaurants, American (New), Italian",Tampa
3,33607,4.0,35,0,"{'BusinessParking': ""{'garage': False, 'street...","Restaurants, Pizza",Tampa
4,33544,4.5,95,1,"{'BestNights': ""{'monday': False, 'tuesday': F...","Burgers, Sports Bars, Bars, Lounges, Restauran...",Wesley Chapel


In [42]:
import ast

df['attributes'] = df['attributes'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

attributes_expanded = pd.json_normalize(df['attributes'])

In [43]:
pd.set_option('display.max_columns', None)

attributes_expanded.head(2)

,Alcohol,OutdoorSeating,RestaurantsReservations,Ambience,GoodForMeal,BusinessParking,RestaurantsGoodForGroups,WiFi,RestaurantsPriceRange2,RestaurantsDelivery,RestaurantsAttire,BusinessAcceptsCreditCards,RestaurantsTakeOut,Caters,NoiseLevel,GoodForKids,BusinessAcceptsBitcoin,BikeParking,RestaurantsTableService,HasTV,WheelchairAccessible,BestNights,ByAppointmentOnly,GoodForDancing,HappyHour,Smoking,CoatCheck,Music,BYOB,DogsAllowed,DriveThru,Corkage,BYOBCorkage,AcceptsInsurance,RestaurantsCounterService,AgesAllowed,Open24Hours,DietaryRestrictions
0,'none',None,False,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...","{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,u'none',False,False,"{'romantic': False, 'intimate': False, 'touris...","{'dessert': False, 'latenight': False, 'lunch'...","{'garage': False, 'street': False, 'validated'...",False,u'no',1,True,u'casual',True,True,True,u'average',True,False,True,False,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:

nested_columns = ['BestNights', 'Ambience', 'Music', 'GoodForMeal', 'BusinessParking', 'DietaryRestrictions']


for column in nested_columns:
    if column in attributes_expanded.columns:
        attributes_expanded[column] = attributes_expanded[column].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

for column in nested_columns:
    if column in attributes_expanded.columns:
        nested_df = pd.json_normalize(attributes_expanded[column]).add_prefix(f"{column}_")
        attributes_expanded = attributes_expanded.drop(columns=[column]).join(nested_df)

attributes_expanded.shape


(8721, 73)

In [48]:
attributes_expanded.head()

,Alcohol,OutdoorSeating,RestaurantsReservations,RestaurantsGoodForGroups,WiFi,RestaurantsPriceRange2,RestaurantsDelivery,RestaurantsAttire,BusinessAcceptsCreditCards,RestaurantsTakeOut,Caters,NoiseLevel,GoodForKids,BusinessAcceptsBitcoin,BikeParking,RestaurantsTableService,HasTV,WheelchairAccessible,ByAppointmentOnly,GoodForDancing,HappyHour,Smoking,CoatCheck,BYOB,DogsAllowed,DriveThru,Corkage,BYOBCorkage,AcceptsInsurance,RestaurantsCounterService,AgesAllowed,Open24Hours,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,Ambience_touristy,Ambience_hipster,Ambience_romantic,Ambience_divey,Ambience_intimate,Ambience_trendy,Ambience_upscale,Ambience_classy,Ambience_casual,Music_dj,Music_background_music,Music_no_music,Music_jukebox,Music_live,Music_video,Music_karaoke,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_brunch,GoodForMeal_breakfast,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian
0,'none',None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,u'none',False,False,False,u'no',1,True,u'casual',True,True,True,u'average',True,False,True,False,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,False,False,False,False,False,False,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,u'full_bar',True,True,True,'no',2,False,u'casual',True,True,True,u'quiet',True,NaN,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,u'beer_and_wine',False,False,True,u'no',1,True,u'casual',True,True,False,u'average',True,NaN,True,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,True,False,False,False,False,False,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,u'full_bar',False,True,True,u'free',2,True,u'casual',True,True,True,u'average',False,False,True,True,True,True,False,False,True,u'outdoor',False,False,False,False,True,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,None,False,False,False,None,None,False,True,False,False,False,None,False,False,None,None,None,None,True,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
attributes_expanded.columns

Index(['Alcohol', 'OutdoorSeating', 'RestaurantsReservations', 'RestaurantsGoodForGroups', 'WiFi',
       'RestaurantsPriceRange2', 'RestaurantsDelivery', 'RestaurantsAttire',
       'BusinessAcceptsCreditCards', 'RestaurantsTakeOut', 'Caters', 'NoiseLevel', 'GoodForKids',
       'BusinessAcceptsBitcoin', 'BikeParking', 'RestaurantsTableService', 'HasTV',
       'WheelchairAccessible', 'ByAppointmentOnly', 'GoodForDancing', 'HappyHour', 'Smoking',
       'CoatCheck', 'BYOB', 'DogsAllowed', 'DriveThru', 'Corkage', 'BYOBCorkage',
       'AcceptsInsurance', 'RestaurantsCounterService', 'AgesAllowed', 'Open24Hours',
       'BestNights_monday', 'BestNights_tuesday', 'BestNights_friday', 'BestNights_wednesday',
       'BestNights_thursday', 'BestNights_sunday', 'BestNights_saturday', 'Ambience_touristy',
       'Ambience_hipster', 'Ambience_romantic', 'Ambience_divey', 'Ambience_intimate',
       'Ambience_trendy', 'Ambience_upscale', 'Ambience_classy', 'Ambience_casual', 'Music_dj',
       

In [50]:
columns_to_clean = ['Alcohol', 'WiFi', 'NoiseLevel', 'Smoking', 'RestaurantsAttire', 'BYOBCorkage', 'AgesAllowed']

for column in columns_to_clean:
    attributes_expanded[column] = attributes_expanded[column].apply(lambda x: x.strip("u'") if isinstance(x, str) else x)


In [51]:
for i in columns_to_clean:
  print(attributes_expanded[i].value_counts())

Alcohol
none             2769
full_bar         2022
beer_and_wine    1785
None                3
Name: count, dtype: int64
WiFi
free    3885
no      2462
paid      48
None       9
Name: count, dtype: int64
NoiseLevel
average      4005
quiet        1241
loud          336
very_loud     111
None            3
Name: count, dtype: int64
Smoking
outdoor    387
no         233
yes         36
None         1
Name: count, dtype: int64
RestaurantsAttire
casual    6140
dressy      92
formal       9
None         6
Name: count, dtype: int64
BYOBCorkage
no             43
yes_free       25
yes_corkage     8
Name: count, dtype: int64
AgesAllowed
allages    11
21plus      1
Name: count, dtype: int64


In [52]:
pd.set_option('display.max_rows', None)

attributes_expanded.isnull().sum().sort_values(ascending=False)

,0
Open24Hours,8720
RestaurantsCounterService,8720
DietaryRestrictions_vegetarian,8719
DietaryRestrictions_soy-free,8719
DietaryRestrictions_halal,8719
DietaryRestrictions_kosher,8719
DietaryRestrictions_vegan,8719
DietaryRestrictions_gluten-free,8719
DietaryRestrictions_dairy-free,8719
AcceptsInsurance,8718


In [53]:
attributes_dummies = pd.get_dummies(attributes_expanded)

In [55]:
attributes_dummies.head()

,Alcohol_None,Alcohol_beer_and_wine,Alcohol_full_bar,Alcohol_none,OutdoorSeating_False,OutdoorSeating_None,OutdoorSeating_True,RestaurantsReservations_False,RestaurantsReservations_None,RestaurantsReservations_True,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_True,WiFi_None,WiFi_free,WiFi_no,WiFi_paid,RestaurantsPriceRange2_1,RestaurantsPriceRange2_2,RestaurantsPriceRange2_3,RestaurantsPriceRange2_4,RestaurantsPriceRange2_None,RestaurantsDelivery_False,RestaurantsDelivery_None,RestaurantsDelivery_True,RestaurantsAttire_None,RestaurantsAttire_casual,RestaurantsAttire_dressy,RestaurantsAttire_formal,BusinessAcceptsCreditCards_False,BusinessAcceptsCreditCards_None,BusinessAcceptsCreditCards_True,RestaurantsTakeOut_False,RestaurantsTakeOut_None,RestaurantsTakeOut_True,Caters_False,Caters_None,Caters_True,NoiseLevel_None,NoiseLevel_average,NoiseLevel_loud,NoiseLevel_quiet,NoiseLevel_very_loud,GoodForKids_False,GoodForKids_None,GoodForKids_True,BusinessAcceptsBitcoin_False,BusinessAcceptsBitcoin_True,BikeParking_False,BikeParking_None,BikeParking_True,RestaurantsTableService_False,RestaurantsTableService_None,RestaurantsTableService_True,HasTV_False,HasTV_None,HasTV_True,WheelchairAccessible_False,WheelchairAccessible_None,WheelchairAccessible_True,ByAppointmentOnly_False,ByAppointmentOnly_None,ByAppointmentOnly_True,GoodForDancing_False,GoodForDancing_True,HappyHour_False,HappyHour_True,Smoking_None,Smoking_no,Smoking_outdoor,Smoking_yes,CoatCheck_False,CoatCheck_True,BYOB_False,BYOB_None,BYOB_True,DogsAllowed_False,DogsAllowed_None,DogsAllowed_True,DriveThru_False,DriveThru_None,DriveThru_True,Corkage_False,Corkage_None,Corkage_True,BYOBCorkage_no,BYOBCorkage_yes_corkage,BYOBCorkage_yes_free,AcceptsInsurance_False,AcceptsInsurance_True,RestaurantsCounterService_True,AgesAllowed_21plus,AgesAllowed_allages,Open24Hours_False,BestNights_monday_False,BestNights_monday_True,BestNights_tuesday_False,BestNights_tuesday_True,BestNights_friday_False,BestNights_friday_True,BestNights_wednesday_False,BestNights_wednesday_True,BestNights_thursday_False,BestNights_thursday_True,BestNights_sunday_False,BestNights_sunday_True,BestNights_saturday_False,BestNights_saturday_True,Ambience_touristy_False,Ambience_touristy_True,Ambience_hipster_False,Ambience_hipster_True,Ambience_romantic_False,Ambience_romantic_True,Ambience_divey_False,Ambience_divey_True,Ambience_intimate_False,Ambience_intimate_True,Ambience_trendy_False,Ambience_trendy_True,Ambience_upscale_False,Ambience_upscale_True,Ambience_classy_False,Ambience_classy_True,Ambience_casual_False,Ambience_casual_True,Music_dj_False,Music_dj_True,Music_background_music_False,Music_background_music_True,Music_no_music_False,Music_jukebox_False,Music_jukebox_True,Music_live_False,Music_live_True,Music_video_False,Music_video_True,Music_karaoke_False,Music_karaoke_True,GoodForMeal_dessert_False,GoodForMeal_dessert_True,GoodForMeal_latenight_False,GoodForMeal_latenight_True,GoodForMeal_lunch_False,GoodForMeal_lunch_True,GoodForMeal_dinner_False,GoodForMeal_dinner_True,GoodForMeal_brunch_False,GoodForMeal_brunch_True,GoodForMeal_breakfast_False,GoodForMeal_breakfast_True,BusinessParking_garage_False,BusinessParking_garage_True,BusinessParking_street_False,BusinessParking_street_True,BusinessParking_validated_False,BusinessParking_validated_True,BusinessParking_lot_False,BusinessParking_lot_True,BusinessParking_valet_False,BusinessParking_valet_True,DietaryRestrictions_dairy-free_False,DietaryRestrictions_gluten-free_True,DietaryRestrictions_vegan_False,DietaryRestrictions_kosher_False,DietaryRestrictions_halal_False,DietaryRestrictions_soy-free_False,DietaryRestrictions_vegetarian_False,DietaryRestrictions_vegetarian_True
0,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [56]:
df = pd.concat([df.drop(columns=['attributes']), attributes_dummies], axis=1)
df.shape

(8721, 174)

In [57]:
categories_expanded = df['categories'].str.get_dummies(sep=', ')
categories_expanded.shape

(8721, 450)

In [58]:
columns_list = categories_expanded.columns.tolist()
print(columns_list)

['Acai Bowls', 'Accessories', 'Accountants', 'Active Life', 'Adult', 'Adult Entertainment', 'Advertising', 'Afghan', 'African', 'Airport Terminals', 'Airports', 'Amateur Sports Teams', 'American (New)', 'American (Traditional)', 'Amusement Parks', 'Appliances', 'Aquariums', 'Arabic', 'Arcades', 'Argentine', 'Armenian', 'Art Classes', 'Art Galleries', 'Art Museums', 'Arts & Crafts', 'Arts & Entertainment', 'Asian Fusion', 'Audio/Visual Equipment Rental', 'Australian', 'Austrian', 'Auto Customization', 'Auto Loan Providers', 'Auto Repair', 'Automotive', 'Bagels', 'Bakeries', 'Bangladeshi', 'Banks & Credit Unions', 'Bar Crawl', 'Barbeque', 'Bars', 'Bartenders', 'Basque', 'Beaches', 'Beauty & Spas', 'Bed & Breakfast', 'Beer', 'Beer Bar', 'Beer Gardens', 'Beer Tours', 'Belgian', 'Beverage Store', 'Bike Rentals', 'Bistros', 'Boat Charters', 'Boat Repair', 'Boating', 'Body Shops', 'Books', 'Bookstores', 'Bowling', 'Brasseries', 'Brazilian', 'Breakfast & Brunch', 'Breweries', 'Brewpubs', 'Brit

In [59]:
non_food_categories = [
    "Accountants", "Advertising", "Appliances", "Aquariums", "Women's Clothing", "Yoga", "Wigs", "Wedding Planning",
    "Water Heater Installation/Repair", "Water Stores", "Web Design", "Videos & Video Game Rental", "Vape Shops",
    "Tutoring Centers", "Tree Services", "Trampoline Parks", "Transportation", "Travel Services", "Tobacco Shops",
    "Thrift Stores", "Ticket Sales", "Tennis", "Team Building Activities", "Swimming Pools", "Specialty Schools",
    "Spiritual Shop", "Sporting Goods", "Skate Shops", "Skating Rinks", "Skin Care", "Real Estate", "Real Estate Law",
    "Race Tracks", "Rafting/Kayaking", "Public Services & Government", "Shared Office Spaces", "Session Photography",
    "Self Storage", "Security Systems", "Screen Printing/T-Shirt Printing", "Religious Organizations",
    "Professional Services", "Propane", "Property Management", "Pressure Washers", "Plastic Surgeons", "Plumbing",
    "Pharmacy", "Photographers", "Photography Stores & Services", "Pest Control", "Pet Boarding", "Pet Groomers",
    "Pet Services", "Pet Sitting", "Pet Stores", "Personal Injury Law", "Pediatricians", "Paddleboarding",
    "Oil Change Stations", "Marketing", "Nail Salons", "Nail Technicians", "Movers", "Mobile Phones",
    "Men's Clothing", "Men's Hair Salons", "Massage", "Massage Therapy", "Mattresses", "Legal Services", "Limos",
    "Lawn Services", "Lawyers", "Medical Spas", "Laser Tag", "Landscape Architects", "Landscaping",
    "Kitchen & Bath", "Home & Garden", "Home Cleaning", "Home Decor", "Home Organization", "Home Services",
    "Homeowner Association", "Junk Removal & Hauling", "Jewelry", "Investing", "Head Shops", "Health & Medical",
    "Heating & Air Conditioning/HVAC", "Hardware Stores", "Haunted Houses", "Hair Loss Centers", "Hair Salons",
    "Go Karts", "Golf", "Golf Equipment", "Golf Lessons", "Graphic Design", "Gemstones & Minerals",
    "General Litigation", "Furniture Stores", "Gas Stations", "Flight Instruction", "Florists", "Flowers & Gifts",
    "Fitness & Instruction", "Financial Services", "Fashion", "Employment Law", "Electronics", "Education",
    "Educational Services", "Drugstores", "Doctors", "Dog Parks", "Divorce & Family Law", "Dance Clubs",
    "Dance Studios", "Day Spas", "Department Stores", "Customized Merchandise", "Cultural Center",
    "Cosmetic Surgeons", "Counseling & Mental Health", "Countertop Installation", "Community Service/Non-Profit",
    "Contract Law", "Contractors", "Churches", "Child Care & Day Care", "Children's Clothing", "Carpet Cleaning",
    "Cannabis Clinics", "Cannabis Dispensaries", "Car Dealers", "Bus Tours", "Business Law", "CSA",
    "Buddhist Temples", "Building Supplies", "Boat Charters", "Boat Repair", "Boating", "Body Shops", "Books",
    "Bookstores", "Bike Rentals", "Beauty & Spas", "Barbers", "Banks & Credit Unions", "Auto Customization",
    "Auto Loan Providers", "Auto Repair", "Automotive", "Audio/Visual Equipment Rental", "Art Classes",
    "Art Galleries", "Art Museums", "Arts & Crafts", "Arts & Entertainment"
]


non_food_set = set(non_food_categories)

filtered_category_df = categories_expanded.drop(columns=non_food_set.intersection(categories_expanded.columns), errors='ignore')

filtered_category_df.shape


(8721, 299)

In [62]:
filtered_category_df.head()

,Acai Bowls,Accessories,Active Life,Adult,Adult Entertainment,Afghan,African,Airport Terminals,Airports,Amateur Sports Teams,American (New),American (Traditional),Amusement Parks,Arabic,Arcades,Argentine,Armenian,Asian Fusion,Australian,Austrian,Bagels,Bakeries,Bangladeshi,Bar Crawl,Barbeque,Bars,Bartenders,Basque,Beaches,Bed & Breakfast,Beer,Beer Bar,Beer Gardens,Beer Tours,Belgian,Beverage Store,Bistros,Bowling,Brasseries,Brazilian,Breakfast & Brunch,Breweries,Brewpubs,British,Bubble Tea,Buffets,Burgers,Burmese,Butcher,Cabaret,Cafes,Cafeteria,Cajun/Creole,Candy Stores,Cantonese,Caribbean,Casinos,Caterers,Champagne Bars,Cheese Shops,Cheesesteaks,Chicken Shop,Chicken Wings,Chinese,Chocolatiers & Shops,Cideries,Cigar Bars,Cinema,Cocktail Bars,Coffee & Tea,Coffee Roasteries,Colombian,Comedy Clubs,Comfort Food,Convenience Stores,Cooking Classes,Cooking Schools,Country Clubs,Couriers & Delivery Services,Creperies,Cuban,Cucina campana,Cupcakes,Custom Cakes,Czech,Delis,Desserts,Dim Sum,Diners,Dinner Theater,Discount Store,Distilleries,Dive Bars,Do-It-Yourself Food,Dominican,Donuts,Drive-Thru Bars,Eatertainment,Egyptian,Empanadas,Ethiopian,Ethnic Food,Ethnic Grocery,Event Planning & Services,Falafel,Farmers Market,Farms,Fast Food,Festivals,Filipino,Fish & Chips,Fishing,Flea Markets,Fondue,Food,Food Court,Food Delivery Services,Food Stands,Food Tours,Food Trucks,French,Fruits & Veggies,Gastropubs,Gay Bars,Gelato,German,Gift Shops,Gluten-Free,Greek,Grilling Equipment,Grocery,Haitian,Halal,Hawaiian,Health Markets,Historical Tours,Hobby Shops,Honduran,Hookah Bars,Hostels,Hot Dogs,Hot Pot,Hotels,Hotels & Travel,Hungarian,Ice Cream & Frozen Yogurt,Imported Food,Indian,Indoor Playcentre,Interior Design,International,International Grocery,Internet Cafes,Irish,Irish Pub,Italian,Izakaya,Japanese,Japanese Curry,Jazz & Blues,Juice Bars & Smoothies,Karaoke,Kebab,Kids Activities,Kombucha,Korean,Kosher,Landmarks & Historical Buildings,Laotian,Latin American,Lebanese,Live/Raw Food,Local Flavor,Local Services,Lounges,Macarons,Mags,Malaysian,Marinas,Meat Shops,Mediterranean,Mexican,Middle Eastern,Mini Golf,Modern European,Mongolian,Moroccan,Museums,Music & Video,Music Venues,Musicians,New Mexican Cuisine,Nightlife,Noodles,Nurseries & Gardening,Nutritionists,Organic Stores,Outlet Stores,Painters,Pakistani,Pan Asian,Parking,Parks,Party & Event Planning,Party Bus Rentals,Party Equipment Rentals,Pasta Shops,Patisserie/Cake Shop,Performing Arts,Persian/Iranian,Personal Chefs,Personal Shopping,Peruvian,Pets,Piano Bars,Pizza,Poke,Polish,Pool & Billiards,Pool Halls,Pop-Up Restaurants,Pop-up Shops,Portuguese,Pretzels,Public Markets,Pubs,Puerto Rican,Ramen,Recreation Centers,Resorts,Restaurant Supplies,Restaurants,Russian,Salad,Salvadoran,Sandwiches,Sardinian,Scandinavian,Scottish,Seafood,Seafood Markets,Senegalese,Shanghainese,Shaved Ice,Shaved Snow,Shopping,Shopping Centers,Sicilian,Smokehouse,Social Clubs,Soul Food,Soup,Southern,Souvenir Shops,Spanish,Speakeasies,Specialty Food,Sports Bars,Sports Clubs,Steakhouses,Street Vendors,Sushi Bars,Syrian,Szechuan,Tacos,Taiwanese,Tapas Bars,Tapas/Small Plates,Tasting Classes,Tea Rooms,Teppanyaki,Tex-Mex,Thai,Themed Cafes,Tiki Bars,Tours,Trinidadian,Turkish,Tuscan,Ukrainian,Vegan,Vegetarian,Venezuelan,Venues & Event Spaces,Vietnamese,Vitamins & Supplements,Waffles,Walking Tours,Whiskey Bars,Wholesale Stores,Wholesalers,Wine & Spirits,Wine Bars,Wine Tasting Classes,Wine Tasting Room,Wine Tours,Wineries,Wraps,Yelp Events
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [63]:
df = pd.concat([df, filtered_category_df], axis=1).drop('categories', axis=1)
df.shape

(8721, 472)

In [64]:
df = pd.concat([df, pd.get_dummies(df['city_updated'])], axis=1).drop('city_updated', axis=1)
df.shape

(8721, 514)

In [74]:
output_filename = "FL_Restaurants_Business Attributes_Edited" + ".csv"
directory = '/content/drive/My Drive/Capstone Data Collection/'

out_path = directory + output_filename

df.to_csv(out_path, index=False)

In [65]:
X = df.drop('stars', axis=1)
y = df['stars']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [67]:
X_train.head()

,postal_code,review_count,is_open,Alcohol_None,Alcohol_beer_and_wine,Alcohol_full_bar,Alcohol_none,OutdoorSeating_False,OutdoorSeating_None,OutdoorSeating_True,RestaurantsReservations_False,RestaurantsReservations_None,RestaurantsReservations_True,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_True,WiFi_None,WiFi_free,WiFi_no,WiFi_paid,RestaurantsPriceRange2_1,RestaurantsPriceRange2_2,RestaurantsPriceRange2_3,RestaurantsPriceRange2_4,RestaurantsPriceRange2_None,RestaurantsDelivery_False,RestaurantsDelivery_None,RestaurantsDelivery_True,RestaurantsAttire_None,RestaurantsAttire_casual,RestaurantsAttire_dressy,RestaurantsAttire_formal,BusinessAcceptsCreditCards_False,BusinessAcceptsCreditCards_None,BusinessAcceptsCreditCards_True,RestaurantsTakeOut_False,RestaurantsTakeOut_None,RestaurantsTakeOut_True,Caters_False,Caters_None,Caters_True,NoiseLevel_None,NoiseLevel_average,NoiseLevel_loud,NoiseLevel_quiet,NoiseLevel_very_loud,GoodForKids_False,GoodForKids_None,GoodForKids_True,BusinessAcceptsBitcoin_False,BusinessAcceptsBitcoin_True,BikeParking_False,BikeParking_None,BikeParking_True,RestaurantsTableService_False,RestaurantsTableService_None,RestaurantsTableService_True,HasTV_False,HasTV_None,HasTV_True,WheelchairAccessible_False,WheelchairAccessible_None,WheelchairAccessible_True,ByAppointmentOnly_False,ByAppointmentOnly_None,ByAppointmentOnly_True,GoodForDancing_False,GoodForDancing_True,HappyHour_False,HappyHour_True,Smoking_None,Smoking_no,Smoking_outdoor,Smoking_yes,CoatCheck_False,CoatCheck_True,BYOB_False,BYOB_None,BYOB_True,DogsAllowed_False,DogsAllowed_None,DogsAllowed_True,DriveThru_False,DriveThru_None,DriveThru_True,Corkage_False,Corkage_None,Corkage_True,BYOBCorkage_no,BYOBCorkage_yes_corkage,BYOBCorkage_yes_free,AcceptsInsurance_False,AcceptsInsurance_True,RestaurantsCounterService_True,AgesAllowed_21plus,AgesAllowed_allages,Open24Hours_False,BestNights_monday_False,BestNights_monday_True,BestNights_tuesday_False,BestNights_tuesday_True,BestNights_friday_False,BestNights_friday_True,BestNights_wednesday_False,BestNights_wednesday_True,BestNights_thursday_False,BestNights_thursday_True,BestNights_sunday_False,BestNights_sunday_True,BestNights_saturday_False,BestNights_saturday_True,Ambience_touristy_False,Ambience_touristy_True,Ambience_hipster_False,Ambience_hipster_True,Ambience_romantic_False,Ambience_romantic_True,Ambience_divey_False,Ambience_divey_True,Ambience_intimate_False,Ambience_intimate_True,Ambience_trendy_False,Ambience_trendy_True,Ambience_upscale_False,Ambience_upscale_True,Ambience_classy_False,Ambience_classy_True,Ambience_casual_False,Ambience_casual_True,Music_dj_False,Music_dj_True,Music_background_music_False,Music_background_music_True,Music_no_music_False,Music_jukebox_False,Music_jukebox_True,Music_live_False,Music_live_True,Music_video_False,Music_video_True,Music_karaoke_False,Music_karaoke_True,GoodForMeal_dessert_False,GoodForMeal_dessert_True,GoodForMeal_latenight_False,GoodForMeal_latenight_True,GoodForMeal_lunch_False,GoodForMeal_lunch_True,GoodForMeal_dinner_False,GoodForMeal_dinner_True,GoodForMeal_brunch_False,GoodForMeal_brunch_True,GoodForMeal_breakfast_False,GoodForMeal_breakfast_True,BusinessParking_garage_False,BusinessParking_garage_True,BusinessParking_street_False,BusinessParking_street_True,BusinessParking_validated_False,BusinessParking_validated_True,BusinessParking_lot_False,BusinessParking_lot_True,BusinessParking_valet_False,BusinessParking_valet_True,DietaryRestrictions_dairy-free_False,DietaryRestrictions_gluten-free_True,DietaryRestrictions_vegan_False,DietaryRestrictions_kosher_False,DietaryRestrictions_halal_False,DietaryRestrictions_soy-free_False,DietaryRestrictions_vegetarian_False,DietaryRestrictions_vegetarian_True,Acai Bowls,Accessories,Active Life,Adult,Adult Entertainment,Afghan,African,Airport Terminals,Airports,Amateur Sports Teams,American (New),American (Traditional),Amusement Parks,Arabic,Arcades,Argentine,Armenian,Asian Fusion,Australian,Aus

Baseline Model

In [72]:

regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

baseline_preds = regression_model.predict(X_test)
valid_ratings = np.array([1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0])
rounded_preds = np.round(baseline_preds * 2) / 2

adjusted_preds = np.array([min(valid_ratings, key=lambda x: abs(x - pred)) for pred in rounded_preds])

y_test_classes = (y_test * 2).astype(int)
adjusted_preds_classes = (adjusted_preds * 2).astype(int)

baseline_accuracy = accuracy_score(y_test_classes, adjusted_preds_classes)
print("Baseline Model - Linear Regression (Rounded to Valid Ratings)")
print(f"Accuracy: {baseline_accuracy:.2f}")
print(f"Mean Squared Error: {mean_squared_error(y_test, adjusted_preds):.2f}")
print(f"R-squared: {r2_score(y_test, adjusted_preds):.2f}")


Baseline Model - Linear Regression (Rounded to Valid Ratings)
Accuracy: 0.30
Mean Squared Error: 0.50
R-squared: 0.30


In [73]:
y_train_class = (y_train * 2).astype(int)
y_test_class = (y_test * 2).astype(int)

classifier_model = RandomForestClassifier()
classifier_model.fit(X_train, y_train_class)

class_preds = classifier_model.predict(X_test)
class_accuracy = accuracy_score(y_test_class, class_preds)
print("Classification Model - Random Forest")
print(f"Accuracy: {class_accuracy:.2f}")
print(classification_report(y_test_class, class_preds))

Classification Model - Random Forest
Accuracy: 0.32
              precision    recall  f1-score   support

           2       0.00      0.00      0.00         9
           3       0.32      0.20      0.24        51
           4       0.25      0.20      0.23        83
           5       0.27      0.16      0.21       176
           6       0.21      0.14      0.17       215
           7       0.31      0.35      0.33       395
           8       0.33      0.48      0.39       438
           9       0.40      0.39      0.39       314
          10       0.09      0.02      0.03        64

    accuracy                           0.32      1745
   macro avg       0.24      0.22      0.22      1745
weighted avg       0.30      0.32      0.31      1745

